# The leaky Integrate-and-Fire (LIF) Neuron

Esta libreta muestra código puro de Pyhton para simular y jugar de manera interactiva con una neurona LIF.

## Las matemáticas del modelo

El modelo de la neurona es el siguiente. Hay una única variable $V$ para la neurona, su potencial de membrana (la diferencia de potencial entre el interior y el exterior de la célula). En ausencia de cualquier entrada, la variable evoluciona con el tiempo de acuerdo con la ecuación diferencial

$$
\tau \frac{\mathrm{d}V}{\mathrm{d}t} = -V.
$$

Aquí la constante $\tau$ es la constante de tiempo de la neurona, y controla la rapidez con que la neurona integra sus entradas. Usted puede resolver fácilmente esta ecuación diferencial para obtener

$$
V(t) = V(0) e^{-t/\tau}.
$$

En otras palabras, $V$ decae exponencialmente hasta el valor 0, y cuanto más pequeño es $\tau$ más rápido lo hace. Puedes comprobarlo con el widget interactivo de abajo.

Si la neurona recibe un pico entrante con un peso sináptico $w$, el potencial de membrana aumenta instantáneamente en $w$, es decir

$$V\leftarrow V+w.$$

En el widget interactivo siguiente, los picos entrantes llegan en los tiempos $t_0$, $t_1$ y $t_2$, que puede cambiar para ver el efecto.

Si uno de los picos entrantes hace que $V$ cruce un valor *umbral*, a veces escrito como $V_t$, entonces la neurona disparará un pico e instantáneamente se restablecerá al valor *restablecido* $V_r$. En ecuaciones:

$$
\text{Si } V>V_t \text{ entonces dispara un pico y } V \leftarrow V_r.
$$

El efecto neto de todo esto es que cuando $\tau$ es grande, la neurona actúa como un *integrador*, sumando sus entradas y disparando cuando alcanzan algún umbral. Si $\tau$ es pequeño, la neurona actúa como un *detector de coincidencias*, disparando un pico sólo si dos o más entradas llegan simultáneamente.

## Detalles de la implementación

Para integrar este código, necesitamos realizar algo parecido al siguiente pseudocódigo:

```
Para cada período de tiempo t:
    Actualizar V del valor en el tiempo t al valor en el tiempo t+dt
    Procesar los picos entrantes
    Comprobar si V ha cruzado el umbral
    En caso afirmativo:
        Emite un pico
        Restablecer V
```

En este caso de una sola neurona, no necesitamos manejar la propagación de la espiga de salida a otras neuronas porque no hay ninguno, sólo registramos que sucedió.

Para actualizar el valor de $V(t)$ a $V(t+\mathrm{d}t)$ utilizamos la ecuación de la sección anterior para obtener

$$V(t+\mathrm{d}t)=V(t)e^{-\mathrm{d}t/\tau}.$$

Nótese que la cantidad $\alpha=e^{-\mathrm{d}t/\tau}$ no depende del tiempo $t$ ni del potencial de membrana $V$, por lo que podemos calcularla una vez fuera del bucle y luego actualizar $V(t+\mathrm{d}t)=\alpha V(t)$.


## Implementación en Python

In [3]:
try:
    import ipywidgets as widgets
except ImportError:
    widgets = None

In [4]:
# Importaciones
import numpy as np
import matplotlib.pyplot as plt

In [18]:
fig = plt.figure(figsize=(5, 4))
ax = fig.add_subplot(111)
plt.close(fig)

'''
    Función que ejecuta la simulación
    Parametros:
        tau: Constante de tiempo (en ms)
        t0, t1, t2: tiempo de los tres pulsos de inicio
        w: peso sináptico de entrada
        threshold: valor de umbral para producir un pulso (spike)
        reset: reinicio de la neurona después de un pulso (spike)
'''
def LIF(tau=10, t0=20, t1=40, t2=60, w=0.1, threshold=1.0, reset=0.0):
    # Tiempos de los picos. Mantener ordenados porque es mas eficiente eliminar el último valor de la lista
    tiempos = [t0, t1, t2]
    tiempos.sort(reverse=True)

    # Definir algunos parámetros por defecto
    duracion = 100 # Tiempo total de la simulación en ms
    dt = 0.1 # Paso de tiempo en ms
    alpha = np.exp(-dt / tau) # Este es el factor por el cual V decae en cada paso de tiempo
    V_rec = [] # Lista para almacenar los potenciales de membrana en cada paso de tiempo
    V = 0.0 # Potencial de membrana inicial
    T = np.arange(np.round(duracion / dt)) * dt # Vector de tiempo
    pulsos = [] # Lista para almacenar los tiempos de los pulsos

    # Ejecutar la simulación
    for t in T:
        V_rec.append(V) # Almacenar el potencial de membrana
        V *= alpha # Integrar ecuaciones
        
        if tiempos and t > tiempos[-1]: # Si ha habido algun pulso
            V += w
            tiempos.pop() # Eliminar ese pulso de la lista
        V_rec.append(V) # Almacenar V antes de resetear, así podemos ver el pulso

        if V > threshold: # Si debería ser un pulso (spike)
            V = reset
            pulsos.append(t)
        
        # Graficar todo (T esta repetido porque almacenamos dos veces V por ciclo)
    ax.clear()
    for t in tiempos:
        ax.axvline(t, ls=':', c='b')
            
    ax.plot(np.repeat(T, 2), V_rec, '-k', lw=2)

    for t in pulsos:
        ax.axvline(t, ls='--', c='r')

    ax.axhline(threshold, ls='--', c='g')
    ax.set_xlim(0, duracion)
    ax.set_ylim(-1, 2)
    ax.set_xlabel('Tiempo (ms)')
    ax.set_ylabel('V')
    plt.tight_layout()
    display(fig);

In [19]:
# Crear un widget para interactuar con la simulación
widgets.interact(LIF, 
                tau=widgets.IntSlider(min=1, max=100, value=50),
                t0=widgets.IntSlider(min=0, max=100, value=20), 
                t1=widgets.IntSlider(min=0, max=100, value=40), 
                t2=widgets.IntSlider(min=0, max=100, value=60), 
                w=widgets.FloatSlider(min=-1.0, max=2.0, step=0.05, value=0.5), 
                threshhold=widgets.FloatSlider(min=0.0, max=2.0, step=0.05, value=1.0),
                reset=widgets.FloatSlider(min=-1.0, max=1.0, step=0.05, value=0.0));

interactive(children=(IntSlider(value=50, description='tau', min=1), IntSlider(value=20, description='t0'), In…